# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xlsxwriter
import math
import yfinance as yf
import pandas_ta
import datetime as dt
import warnings
from scipy.stats import percentileofscore as score

warnings.filterwarnings('ignore') 

## Importing Our List of Stocks

In [2]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [90]:
msft = yf.Ticker("SJM")
hist = msft.history(period="1d")
info = msft.info
price = info["currentPrice"]
EPS = info["trailingEps"]
PE = round(price / EPS, 2)
# price
PE

# # hist
# trailing_pe = info['trailingPE']
# forward_pe = info['forwardPE']

# trailing_pe
# # forward_pe
# price = hist['Close'][0]

info

{'address1': 'One Strawberry Lane',
 'city': 'Orrville',
 'state': 'OH',
 'zip': '44667-0280',
 'country': 'United States',
 'phone': '330 682 3000',
 'fax': '330 684 3370',
 'website': 'https://www.jmsmucker.com',
 'industry': 'Packaged Foods',
 'industryKey': 'packaged-foods',
 'industryDisp': 'Packaged Foods',
 'sector': 'Consumer Defensive',
 'sectorKey': 'consumer-defensive',
 'sectorDisp': 'Consumer Defensive',
 'longBusinessSummary': "The J. M. Smucker Company manufactures and markets branded food and beverage products worldwide. It operates in three segments: U.S. Retail Pet Foods, U.S. Retail Coffee, and U.S. Retail Consumer Foods. The company offers mainstream roast, ground, single serve, and premium coffee; peanut butter and specialty spreads; fruit spreads, toppings, and syrups; jelly products; nut mix products; shortening and oils; frozen sandwiches and snacks; pet food and pet snacks; and foodservice hot beverage, foodservice portion control, and flour products, as well a

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [48]:
cols = ['Ticker', 'Price', 'Trailing PE Ratio', 'Forward PE Ratio', 'Number of Shares to Buy']
res_df = pd.DataFrame(columns = cols)
res_df

,Ticker,Price,Trailing PE Ratio,Forward PE Ratio,Number of Shares to Buy


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [49]:
symbols_string = " ".join(str(element) for element in symbols_list)
tickers = yf.Tickers(symbols_string)

symbols_string

'MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BK BBWI BAX BDX BRK-B BBY BIO TECH BIIB BLK BX BA BKNG BWA BXP BSX BMY AVGO BR BRO BF-B BLDR BG CDNS CZR CPT CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE COR CNC CNP CDAY CF CHRW CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ CEG COO CPRT GLW CTVA CSGP COST CTRA CCI CSX CMI CVS DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DG DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV LLY EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ESS EL ETSY EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FLT FMC F FTNT FTV FOXA FOX BEN FCX GRMN IT GEHC GEN GNRC GD GE GIS GM GPC GILD GPN GL GS HAL HIG HAS HCA PEAK HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM HPQ HUB

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [93]:
res_df = pd.DataFrame(columns = cols)

for symbol in symbols_list:
    info = tickers.tickers[symbol].info
    price = info["currentPrice"]
    
    try:
        trailing_pe = float(info['trailingPE'])
    # except KeyError:
    #     try:
    #         trailing_eps = float(info["trailingEps"])
    #         trailing_pe = round(price / trailing_eps, 2)
    #     except:
    #         trailing_pe = None
    # except ValueError: 
    #     trailing_pe = None
    except:
        trailing_pe = None

    try:
        forward_pe = float(info['forwardPE'])
    # except KeyError:
    #     try: 
    #         forward_eps = float(info['forwardEps'])
    #         forward_pe = round(price / forward_eps, 2)
    #     except:
    #         forward_pe = None
    # except ValueError:
    #         forward_pe = None
    except: 
        forward_pe = None

    res_df.loc[len(res_df)] = [symbol, price, trailing_pe, forward_pe, 'N/A']

res_df

,Ticker,Price,Trailing PE Ratio,Forward PE Ratio,Number of Shares to Buy
0,MMM,109.69,NaN,11.181447,N/A
1,AOS,82.24,40.915420,20.508726,N/A
2,ABT,110.40,37.551020,23.844492,N/A
3,ABBV,154.75,42.281420,13.866487,N/A
4,ACN,351.59,32.584800,26.257654,N/A
...,...,...,...,...,...
498,YUM,130.52,24.813688,22.387651,N/A
499,ZBRA,275.35,30.799778,24.540998,N/A
500,ZBH,121.63,53.581497,15.318640,N/A
501,ZION,44.66,8.270371,10.972973,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [94]:
# with pd.option_context('mode.use_inf_as_null', True):
# res_df['Forward PE Ratio'] = res_df['Forward PE Ratio'].astype(float)

res_df.sort_values('Trailing PE Ratio', inplace=True, na_position='last')
res_df = res_df[res_df['Trailing PE Ratio'] > 0]
res_df = res_df[res_df['Forward PE Ratio'] > 0]
res_df.dropna(inplace=True)

res_df = res_df[:50]
res_df.reset_index(inplace = True)
res_df.drop('index', axis=1, inplace = True)

res_df

,Ticker,Price,Trailing PE Ratio,Forward PE Ratio,Number of Shares to Buy
0,VLO,130.09,4.436903,9.027759,N/A
1,UAL,41.97,4.857639,4.390167,N/A
2,GM,36.20,5.084270,5.216138,N/A
3,ALB,149.43,5.297058,11.363498,N/A
4,EQT,39.02,5.345205,11.544378,N/A
5,MPC,146.88,5.540551,9.593730,N/A
6,AAL,13.98,5.849372,6.625593,N/A
7,CMA,56.57,6.406568,9.942003,N/A
8,EG,352.61,6.516540,5.721402,N/A
9,SYF,38.37,7.066299,7.158582,N/A


In [88]:
# def pe_difference(row):
#     return row['Forward PE Ratio'] - row['Trailing PE Ratio'];

# res_df['PE Difference'] = res_df.apply(pe_difference, axis=1)

# res_df

,Ticker,Price,Trailing PE Ratio,Forward PE Ratio,Number of Shares to Buy,PE Difference
148,DLR,136.12,47.100340,116.341880,N/A,69.241540
456,UDR,38.77,28.094204,94.560974,N/A,66.466770
251,PODD,219.43,126.838140,89.930320,N/A,-36.907820
486,WELL,91.16,172.000020,75.966670,N/A,-96.033350
176,EQIX,814.09,87.630790,74.755740,N/A,-12.875050
...,...,...,...,...,...,...
143,DAL,40.63,7.666038,6.184170,N/A,-1.481867
181,EG,352.61,6.516540,5.721402,N/A,-0.795138
217,GM,36.20,5.084270,5.216138,N/A,0.131868
459,UAL,41.97,4.857639,4.390167,N/A,-0.467472


In [89]:
# res_df.sort_values('PE Difference', inplace=True, na_position='last')
# res_df = res_df[:50]
# res_df.reset_index(inplace = True)
# res_df.drop('index', axis=1, inplace = True)

# res_df

,Ticker,Price,Trailing PE Ratio,Forward PE Ratio,Number of Shares to Buy,PE Difference
0,SJM,125.78,12578.000000,12.319295,N/A,-12565.680705
1,CDAY,67.99,2266.333300,40.230766,N/A,-2226.102534
2,IFF,81.85,1364.166600,20.209875,N/A,-1343.956725
3,AMD,148.76,1239.666600,44.011830,N/A,-1195.654770
4,WYNN,91.76,327.714300,19.648823,N/A,-308.065477
5,CLX,142.06,205.884050,23.835570,N/A,-182.048480
6,CAH,100.28,164.393430,13.057292,N/A,-151.336138
7,PANW,295.58,165.128500,46.040497,N/A,-119.088003
8,AMT,217.16,144.773330,44.228107,N/A,-100.545223
9,WELL,91.16,172.000020,75.966670,N/A,-96.033350


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [96]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [97]:
portfolio_input()

Enter the value of your portfolio: 1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [105]:
position_size = float(portfolio_size)/len(res_df.index)
for row in res_df.index:
    res_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/res_df.loc[row, 'Price'])

res_df

,Ticker,Price,Trailing PE Ratio,Forward PE Ratio,Number of Shares to Buy
0,VLO,130.09,4.436903,9.027759,153
1,UAL,41.97,4.857639,4.390167,476
2,GM,36.20,5.084270,5.216138,552
3,ALB,149.43,5.297058,11.363498,133
4,EQT,39.02,5.345205,11.544378,512
5,MPC,146.88,5.540551,9.593730,136
6,AAL,13.98,5.849372,6.625593,1430
7,CMA,56.57,6.406568,9.942003,353
8,EG,352.61,6.516540,5.721402,56
9,SYF,38.37,7.066299,7.158582,521


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [109]:
symbol = 'AAPL'
info = yf.Ticker(symbol).info

info

{'address1': 'One Apple Park Way',
 'city': 'Cupertino',
 'state': 'CA',
 'zip': '95014',
 'country': 'United States',
 'phone': '408 996 1010',
 'website': 'https://www.apple.com',
 'industry': 'Consumer Electronics',
 'industryKey': 'consumer-electronics',
 'industryDisp': 'Consumer Electronics',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and pod

In [112]:
pe = info['trailingPE']
pb = info['priceToBook']
ps = info['priceToSalesTrailing12Months']
ev_ebita = ['enterpriseToEbitda']
ev_gp = float(info['enterpriseValue']) / float(info['grossProfits'])

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [125]:
rv_col = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'PE Ratio',
    'PE Percentile',
    'PB Ratio',
    'PB Percentile',
    'PS Ratio',
    'PS Percentile',
    'EV/EBITA Ratio',
    'EV/EBITA Percentile',
    'EV/GP Ratio',
    'EV/GP Percentile',
    'RV Score'
]

rv_df = pd.DataFrame(columns = rv_col)
rv_df

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITA Ratio,EV/EBITA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score


In [130]:
rv_df = pd.DataFrame(columns = rv_col)

for symbol in symbols_list:
    info = yf.Ticker(symbol).info
    price = info["currentPrice"]
    try:
        pe = info['trailingPE']
    except:
        pe = np.NaN

    try:
        pb = info['priceToBook']
    except:
        pb = np.NaN

    try:
        ps = info['priceToSalesTrailing12Months']
    except:
        ps = np.NaN

    try:
        ev_ebita = info['enterpriseToEbitda']
    except:
        ev_ebita = np.NaN

    try:
        ev_gp = float(info['enterpriseValue']) / float(info['grossProfits'])
    except:
        ev_gp = np.NaN
    rv_df.loc[len(rv_df)] = [symbol, price, np.NaN, pe, np.NaN, pb, np.NaN, ps, np.NaN, ev_ebita, np.NaN, ev_gp, np.NaN, np.NaN]
    
rv_df

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITA Ratio,EV/EBITA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,MMM,109.32,NaN,NaN,NaN,12.923514,NaN,1.843811,NaN,9.266,NaN,4.809487,NaN,NaN
1,AOS,82.44,NaN,41.014927,NaN,6.513392,NaN,3.225762,NaN,14.953,NaN,9.088231,NaN,NaN
2,ABT,110.07,NaN,37.438774,NaN,5.098194,NaN,4.782105,NaN,20.257,NaN,8.135114,NaN,NaN
3,ABBV,154.97,NaN,42.341530,NaN,22.623358,NaN,4.962199,NaN,11.486,NaN,7.742730,NaN,NaN
4,ACN,350.91,NaN,32.582172,NaN,8.247585,NaN,3.406468,NaN,19.176,NaN,10.455590,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.66,NaN,24.793169,NaN,NaN,NaN,5.188418,NaN,19.263,NaN,14.579701,NaN,NaN
499,ZBRA,273.33,NaN,30.573826,NaN,4.659166,NaN,2.764498,NaN,17.432,NaN,6.264528,NaN,NaN
500,ZBH,121.70,NaN,53.849556,NaN,2.033077,NaN,3.493926,NaN,12.942,NaN,6.269094,NaN,NaN
501,ZION,43.87,NaN,8.124074,NaN,1.333151,NaN,2.108792,NaN,NaN,NaN,2.654881,NaN,NaN


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [135]:
for col in ['PE Ratio', 'PB Ratio', 'PS Ratio', 'EV/EBITA Ratio', 'EV/GP Ratio']:
    rv_df[col].fillna(rv_df[col].mean(), inplace = True)

rv_df

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITA Ratio,EV/EBITA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,MMM,109.32,NaN,73.839564,NaN,12.923514,NaN,1.843811,NaN,9.266000,NaN,4.809487,NaN,NaN
1,AOS,82.44,NaN,41.014927,NaN,6.513392,NaN,3.225762,NaN,14.953000,NaN,9.088231,NaN,NaN
2,ABT,110.07,NaN,37.438774,NaN,5.098194,NaN,4.782105,NaN,20.257000,NaN,8.135114,NaN,NaN
3,ABBV,154.97,NaN,42.341530,NaN,22.623358,NaN,4.962199,NaN,11.486000,NaN,7.742730,NaN,NaN
4,ACN,350.91,NaN,32.582172,NaN,8.247585,NaN,3.406468,NaN,19.176000,NaN,10.455590,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.66,NaN,24.793169,NaN,8.910963,NaN,5.188418,NaN,19.263000,NaN,14.579701,NaN,NaN
499,ZBRA,273.33,NaN,30.573826,NaN,4.659166,NaN,2.764498,NaN,17.432000,NaN,6.264528,NaN,NaN
500,ZBH,121.70,NaN,53.849556,NaN,2.033077,NaN,3.493926,NaN,12.942000,NaN,6.269094,NaN,NaN
501,ZION,43.87,NaN,8.124074,NaN,1.333151,NaN,2.108792,NaN,18.310811,NaN,2.654881,NaN,NaN


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [143]:
metrics = {
    'PE Ratio': 'PE Percentile',
    'PB Ratio': 'PB Percentile',
    'PS Ratio': 'PS Percentile',
    'EV/EBITA Ratio': 'EV/EBITA Percentile',
    'EV/GP Ratio': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[metric]] = score(rv_df[metric], rv_df.loc[row, metric])

rv_df

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITA Ratio,EV/EBITA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,MMM,109.32,NaN,73.839564,90.357853,12.923514,90.059642,1.843811,32.007952,9.266000,15.705765,4.809487,20.278330,NaN
1,AOS,82.44,NaN,41.014927,75.546720,6.513392,68.190855,3.225762,56.262425,14.953000,44.731610,9.088231,48.111332,NaN
2,ABT,110.07,NaN,37.438774,71.371769,5.098194,60.039761,4.782105,71.371769,20.257000,75.745527,8.135114,42.942346,NaN
3,ABBV,154.97,NaN,42.341530,76.540755,22.623358,95.427435,4.962199,73.359841,11.486000,27.037773,7.742730,40.357853,NaN
4,ACN,350.91,NaN,32.582172,65.606362,8.247585,74.353877,3.406468,57.057654,19.176000,71.371769,10.455590,62.027833,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.66,NaN,24.793169,45.725646,8.910963,78.926441,5.188418,74.751491,19.263000,71.769384,14.579701,79.721670,NaN
499,ZBRA,273.33,NaN,30.573826,62.425447,4.659166,57.455268,2.764498,50.695825,17.432000,56.858847,6.264528,31.013917,NaN
500,ZBH,121.70,NaN,53.849556,82.504970,2.033077,24.850895,3.493926,59.045726,12.942000,35.785288,6.269094,31.411531,NaN
501,ZION,43.87,NaN,8.124074,4.572565,1.333151,8.747515,2.108792,37.773360,18.310811,65.109344,2.654881,6.361829,NaN


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [146]:
from statistics import mean

for row in rv_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_df.loc[row, metrics[metric]])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)

rv_df

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITA Ratio,EV/EBITA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,MMM,109.32,NaN,73.839564,90.357853,12.923514,90.059642,1.843811,32.007952,9.266000,15.705765,4.809487,20.278330,49.681909
1,AOS,82.44,NaN,41.014927,75.546720,6.513392,68.190855,3.225762,56.262425,14.953000,44.731610,9.088231,48.111332,58.568588
2,ABT,110.07,NaN,37.438774,71.371769,5.098194,60.039761,4.782105,71.371769,20.257000,75.745527,8.135114,42.942346,64.294235
3,ABBV,154.97,NaN,42.341530,76.540755,22.623358,95.427435,4.962199,73.359841,11.486000,27.037773,7.742730,40.357853,62.544732
4,ACN,350.91,NaN,32.582172,65.606362,8.247585,74.353877,3.406468,57.057654,19.176000,71.371769,10.455590,62.027833,66.083499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.66,NaN,24.793169,45.725646,8.910963,78.926441,5.188418,74.751491,19.263000,71.769384,14.579701,79.721670,70.178926
499,ZBRA,273.33,NaN,30.573826,62.425447,4.659166,57.455268,2.764498,50.695825,17.432000,56.858847,6.264528,31.013917,51.689861
500,ZBH,121.70,NaN,53.849556,82.504970,2.033077,24.850895,3.493926,59.045726,12.942000,35.785288,6.269094,31.411531,46.719682
501,ZION,43.87,NaN,8.124074,4.572565,1.333151,8.747515,2.108792,37.773360,18.310811,65.109344,2.654881,6.361829,24.512922


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [148]:
rv_df.sort_values('RV Score', ascending = True, inplace = True)

rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace = True)

rv_df

,Ticker,Price,Number of Shares to Buy,PE Ratio,PE Percentile,PB Ratio,PB Percentile,PS Ratio,PS Percentile,EV/EBITA Ratio,EV/EBITA Percentile,EV/GP Ratio,EV/GP Percentile,RV Score
0,UAL,41.26,NaN,4.775463,0.397614,1.528714,13.121272,0.257833,1.590457,4.107000,1.988072,2.514325,5.765408,4.572565
1,VLO,130.00,NaN,4.433834,0.198807,1.703912,17.892644,0.304279,2.584493,2.988000,1.192843,2.702397,6.560636,5.685885
2,MOS,35.73,NaN,9.045569,7.554672,0.973543,2.584493,0.777076,9.940358,5.415000,3.777336,2.618482,6.163022,6.003976
3,VTRS,10.83,NaN,7.172185,2.385686,0.622664,0.994036,0.840086,10.735586,6.128000,5.367793,4.298587,17.296223,7.355865
4,AIG,67.75,NaN,11.762153,13.320080,1.208505,6.163022,0.948400,12.326044,5.102000,3.180915,2.414860,4.970179,7.992048
5,BG,100.95,NaN,7.813467,3.578529,1.441402,11.133201,0.239404,1.391650,5.517000,4.174950,4.915983,20.874751,8.230616
6,BWA,35.85,NaN,8.743902,6.163022,1.460940,11.729622,0.491096,5.367793,4.782000,2.982107,3.733227,14.910537,8.230616
7,HPE,16.98,NaN,11.025974,11.729622,1.028468,3.180915,0.757645,9.343936,6.299000,6.163022,3.390070,12.127237,8.508946
8,MPC,148.36,NaN,5.596379,1.192843,2.214328,29.224652,0.368835,3.379722,4.071000,1.789264,2.966117,7.753479,8.667992
9,CNC,74.21,NaN,16.454544,25.248509,1.568391,14.314115,0.284457,1.988072,5.871000,5.168986,1.570905,2.783300,9.900596


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [ ]:
portfolio_input()

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HPE,9.830,1994,-293.68,0.0118812,0.768699,0.134653,4.757379,0.0455446,2.575722,0.039604,0.0579208
1,FTI,8.870,2210,-0.68,0.134653,0.526579,0.0732673,2.722313,0.00792079,1.745265,0.0178218,0.0584158
2,AAL,13.360,1467,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
3,CCL,15.610,1256,-3.90,0.10297,0.380999,0.0554455,3.751649,0.0237624,3.458441,0.0811881,0.0658416
4,HFC,25.660,764,-26.18,0.0435644,0.716944,0.120792,3.301222,0.0118812,3.693071,0.0891089,0.0663366
5,HPQ,19.140,1024,9.17,0.172277,-23.323972,0.0237624,5.822424,0.0673267,2.516859,0.0356436,0.0747525
6,XRX,17.800,1101,9.35,0.174257,0.640297,0.0990099,3.620185,0.019802,1.604068,0.0138614,0.0767327
7,TPR,16.010,1224,-20.58,0.049505,1.200922,0.231683,3.870569,0.029703,1.149885,0.0019802,0.0782178
8,LB,27.330,717,-10.49,0.0633663,-5.047485,0.0455446,8.144139,0.170297,3.119623,0.0613861,0.0851485
9,SYF,25.300,775,7.77,0.152475,0.971730,0.180198,2.129599,0.0039604,1.604512,0.0158416,0.0881188


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: